# AgentCore Gateway ターゲットに対する Authorization Code Grant タイプのアウトバウンド OAuth フローの使用

## 概要
アウトバウンド認証/認可により、Amazon Bedrock AgentCore ゲートウェイは、インバウンド認可中に認証・認可されたユーザーに代わって、ゲートウェイターゲットに安全にアクセスできます。AgentCore Gateway は、認可なし、IAM ベースのアウトバウンド認可、OAuth、API キーなど、複数のタイプのアウトバウンド認可をサポートしています。

以下のタイプの OAuth 認可グラントを使用できます：
* **Client credentials grant** – マシン間認証（2-legged OAuth とも呼ばれます）。クライアントアプリケーションは、ユーザーに代わってではなく、アプリケーション自体に代わってリソースにアクセスします。
* **Authorization code grant（新機能）** – ユーザー委任アクセス（3-legged OAuth とも呼ばれます）。ユーザーは、クライアントアプリケーションがユーザーに代わってリソースにアクセスすることに同意します。

Model Context Protocol（MCP）仕様の **バージョン 2025-11-25** では、MCP は [URL Mode Elicitation](https://blog.modelcontextprotocol.io/posts/2025-11-25-first-mcp-anniversary/#url-mode-elicitation-secure-out-of-band-interactions) をサポートしています。URL モード Elicitation により、ユーザーをブラウザ内の適切な OAuth フロー（または任意の資格情報取得フロー）に誘導し、入力された資格情報をクライアントが見ることなく安全に認証できます。資格情報はサーバーによって直接管理され、クライアントはサーバーへの独自の認可フローのみを処理すれば済みます。

AgentCore Gateway は、MCP バージョン 2025-11-25 をサポートするゲートウェイのアウトバウンド認証として **Authorization code grant** をサポートするようになりました。Authorization code grant は、Google（メール、カレンダーなど）、Github（リポジトリ、プルリクエストなど）、Linkedin（ユーザープロフィール、投稿など）、または内部ツールなど、ユーザーに代わってツールにアクセスする場合に便利です。Authorization code grant フローは、ユーザーの資格情報をサードパーティアプリケーションに公開しないことで、ユーザーのプライバシーを優先します。

このチュートリアルでは、Authorization code grant を使用して Linkedin ツールをターゲットとする AgentCore Gateway を作成します。作成された AgentCore Gateway は、ユーザーに代わってユーザー情報の取得、投稿の読み取り/要約、または新しい投稿の作成など、Linkedin アクションを自動化するエージェントの構築に使用できます。


## 適切な認証/認可パターンの選択
エージェントの認証/認可戦略を設計する際、どのパターンが最適かを判断するために以下の要素を考慮してください：

| **要素** | **OAuth 2.0 authorization code grant（ユーザー委任アクセス）** | **OAuth 2.0 client credentials grant（マシン間認証）** |
|----------|----------------------------------------------------------------|-----------------------------------------------------------------------------|
| **データ所有権** | ユーザー固有のデータ（メール、ドキュメント、個人カレンダー） | システムまたは組織所有のデータ（分析、ログ、共有リソース） |
| **ユーザーインタラクション** | ユーザーが存在し、同意を提供可能 | ユーザーインタラクションは不要または利用不可 |
| **操作タイミング** | インタラクティブなリアルタイム操作 | バックグラウンド、スケジュール、またはバッチ操作 |
| **権限スコープ** | 権限はユーザーと同意の選択によって異なる | エージェントレベルで定義された一貫した権限 |

## Authorization code grant の主な特徴

* 認可プロンプトを通じた明示的なユーザー同意が必要
* ユーザー固有のデータとリソースへのアクセスを提供
* エージェントの ID とユーザー認可の間の明確な分離を維持
* エージェントがアクセスできるデータを制限するきめ細かいスコープをサポート

### チュートリアルアーキテクチャ

<div style="text-align:center">
    <img src="images/outbound_auth_3lo.png" width="90%"/>
</div>


### チュートリアル詳細

| 情報                | 詳細                                                                     |
|:--------------------|:-------------------------------------------------------------------------|
| チュートリアルタイプ | インタラクティブ                                                         |
| エージェントタイプ   | MCP Client                                                               |
| エージェントフレームワーク | MCP SDK                                                             |
| LLM モデル          | N/A                                                                      |
| チュートリアルコンポーネント | Linkedin ターゲットを持つ AgentCore Gateway（OAuth 2.0 authorization code grant 使用） |
| チュートリアル業種   | クロスバーティカル                                                       |
| 難易度              | 中                                                                       |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK と Boto3                              |
| 資格情報プロバイダー | タイプ：OAuth2 - Linkedin Provider                                       |

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 認証情報
* UV

In [ ]:
# Install from the requirements file in current directory
!uv pip install --system --force-reinstall --no-cache-dir -r requirements.txt --quiet

In [ ]:
# Import required libraries and generate a unique timestamp for naming resources.

import boto3
import json
import time
import zipfile
import io
import os
import sys
import requests
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from botocore.exceptions import ClientError
from datetime import datetime

print("ライブラリをインポートしました")

# Generate timestamp for unique naming
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
print(f"タイムスタンプ: {timestamp}")

REGION = os.environ['AWS_REGION']

gateway_target_name = f"mcp-target-{timestamp}"

In [ ]:
# Import Utilities and Configure Logging

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now import utils
import utils

# Setup logging 
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

logging.getLogger("strands").setLevel(logging.INFO)

print("ロギング設定完了、utils をインポートしました")

## Amazon Cognito を IDP としたインバウンド認証の設定

アプリクライアントを持つ Cognito User Pool をプロビジョニングします。Amazon Cognito を使用して、デプロイされた MCP サーバーにアクセスするための JWT トークンを提供します。

In [ ]:
USER_POOL_NAME = f"agentcore-gateway-authcode-pool-{timestamp}"
RESOURCE_SERVER_ID = f"agentcore-gateway-authcode-id-{timestamp}"
RESOURCE_SERVER_NAME = f"agentcore-gateway-authcode-name-{timestamp}"
CLIENT_NAME = f"agentcore-gateway-authcode-client-{timestamp}"

# Scopes are based on the current gateway_target_name for THIS run
SCOPES = [
    # Full access to MCP target
    {
        "ScopeName": gateway_target_name,
        "ScopeDescription": "Full access to all tools in MCP target"
    }
]

# Full scope strings in Cognito format: "<resource-server-id>/<scope-name>"
scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
scopeString = " ".join(scope_names)

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognito リソースを作成または取得中...")
gw_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {gw_user_pool_id}")

utils.get_or_create_resource_server(
    cognito,
    gw_user_pool_id,
    RESOURCE_SERVER_ID,
    RESOURCE_SERVER_NAME,
    SCOPES
)
print("リソースサーバーを確認しました。")

gw_client_id, gw_client_secret = utils.get_or_create_m2m_client(
    cognito,
    gw_user_pool_id,
    CLIENT_NAME,
    RESOURCE_SERVER_ID,
    scope_names
)
print(f"Client ID: {gw_client_id}")

# Discovery URL used later by the Gateway authorizer and utils.get_token
gw_cognito_discovery_url = (
    f"https://cognito-idp.{REGION}.amazonaws.com/{gw_user_pool_id}/.well-known/openid-configuration"
)
print(gw_cognito_discovery_url)

## アウトバウンド OAuth2 用の Linkedin 資格情報プロバイダーの設定

### ステップ 1: LinkedIn Client Id と Client Secret の取得

LinkedIn API を使用して Authorization Code Grant フローをテストします。まず、アクセストークンを取得するために LinkedIn の ClientId と ClientSecret を取得する必要があります。取得するには：

* https://developer.linkedin.com/ にアクセスします
* アプリを作成をクリックし、アプリ名を追加します
* LinkedIn ページの作成が必須ステップとして必要です。For Business をクリックすると、ページを作成するオプションが表示されます。任意のダミーページを作成してください。
* アプリを作成すると、開発者ポータルの My Apps セクションに表示されます
* アプリの Auth セクションで、アクセストークンの取得に使用できる clientId と clientSecret が見つかります。
* アプリの Products セクションで、アクセス可能な API を有効にする必要があります。「Sign In with LinkedIn using OpenID Connect」の「Request Access」をクリックします。
* これにより、ユーザープロフィール情報を取得する API が有効になります。

<div style="text-align:center">
    <img src="images/linkedin-oauth1.png" width="90%"/>
</div>

<div style="text-align:center">
    <img src="images/linkedin-oauth2.png" width="90%"/>
</div>

注意：OAuth 2.0 settings というセクションもあり、Authorized Redirect URLs を提供する必要があります。次のステップで資格情報プロバイダーを作成した後、このセクションに戻ります。

### ステップ 2: AgentCore Identity で Linkedin 資格情報プロバイダーを作成
Amazon Bedrock AgentCore Identity は、インバウンドとアウトバウンドの両方の認証に対して、マネージド OAuth 2.0 対応プロバイダーを提供します。各プロバイダーは、特定のサービスまたは ID システムに必要な特定の認証プロトコル、エンドポイント設定、および資格情報形式をカプセル化します。このサービスは、Google、GitHub、Slack、Salesforce などの人気サービス向けに、認可サーバーエンドポイントとプロバイダー固有のパラメータが事前設定されたビルトインプロバイダーを提供し、開発労力を削減します。プロバイダーは、異なる OAuth 2.0 実装、API 認証スキーム、およびトークン形式の複雑さを抽象化し、基礎となるプロトコルの変動やエッジケースを処理しながら、エージェントに統一されたインターフェースを提供します。

このチュートリアルでは、Linkedin をビルトインプロバイダーとして使用する資格情報プロバイダーを作成します。

**先ほど控えた Linkedin Client Id と Client Secret を入力してください。**

In [ ]:
target_client_id = "<clientid>" #replace this with client id from https://developer.linkedin.com/
target_client_secret = "<clientsecret>" #replace this with client secret from https://developer.linkedin.com/

target_cred_provider_name = f"ac-gateway-mcp-server-identity-authcode-{timestamp}"

identity_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

print(f"名前 {target_cred_provider_name} の資格情報プロバイダーが既に存在する場合は削除します")
try:
    delete_resp = identity_client.delete_oauth2_credential_provider(name=target_cred_provider_name)
    print("既存の資格情報プロバイダーが見つかり削除しました。再作成を続行します")
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceNotFound':
        print("同じ名前の既存の資格情報プロバイダーは存在しません。作成を続行します")
    else:
        raise Exception(f"資格情報プロバイダーの削除に失敗しました: {e.response['Error']['Message']}")

linkedin_cred_provider = identity_client.create_oauth2_credential_provider(
    name=target_cred_provider_name,
    credentialProviderVendor="LinkedinOauth2",
    oauth2ProviderConfigInput={
        'linkedinOauth2ProviderConfig': {
            'clientId': target_client_id,
            'clientSecret': target_client_secret
        }
    }
)

target_cred_provider_arn = linkedin_cred_provider['credentialProviderArn']
target_callback_url = linkedin_cred_provider['callbackUrl']
print("アウトバウンド OAuth2 資格情報プロバイダー ARN:", target_cred_provider_arn)
print("以下のコールバック URL を Linkedin に登録してください:", target_callback_url)

### ステップ 3: コールバック URL を Linkedin に登録
前のセルの出力として受け取ったコールバック URL を Linkedin アプリに登録します。

<div style="text-align:center">
    <img src="images/linkedin-oauth3.png" width="90%"/>
</div>

## Cognito をインバウンド認証として AgentCore Gateway を作成
Cognito をインバウンド認証プロバイダーとして AgentCore Gateway を作成します。

In [ ]:
def create_gateway_with_authcode():
    iam_client = boto3.client('iam', region_name=REGION)
    gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
    
    role_name = f'BedrockAgentCoreGatewayRole-{timestamp}'
    gateway_name = f"gateway-authcode-{timestamp}"
    
    # Create IAM role for Gateway
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock-agentcore.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    try:
        iam_response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description='IAM role for Bedrock Agent Core Gateway with 3LO'
        )

        role_arn = iam_response['Role']['Arn']
        print(f"Gateway IAM ロールを作成しました: {role_arn}")

        # Attach admin policy to the role
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
        )
        print("Gateway IAM ロールに管理者ポリシーをアタッチしました")
    except ClientError as e:
        if e.response['Error']['Code'] == 'EntityAlreadyExists':
            iam_response = iam_client.get_role(RoleName=role_name)
            role_arn = iam_response['Role']['Arn']
            print(f"IAM ロールは既に存在します（ARN: {role_arn}）。既存のロールを使用します")
        else:
            raise Exception(f"IAM ロールの作成に失敗しました: {e.response['Error']['Message']}")

    print("Auth Code grant を使用して Gateway を作成中...")
    gateway_response = gateway_client.create_gateway(
        name=gateway_name,
        protocolType="MCP",
        protocolConfiguration={
            "mcp": {
                "supportedVersions": ["2025-03-26", "2025-11-25"],
                "searchType": "SEMANTIC"
            }
        },
       authorizerType="CUSTOM_JWT",
       authorizerConfiguration={
            "customJWTAuthorizer": {
                "discoveryUrl": gw_cognito_discovery_url,
                "allowedClients": [gw_client_id]
            }
       },
       roleArn=role_arn 
    )

    print("Gateway 作成レスポンス:", gateway_response)

    gateway_id = gateway_response['gatewayId']
    gateway_url = gateway_response['gatewayUrl']
    print(f"Auth Code grant を使用した Gateway を作成しました: {gateway_id}")

    # Wait for gateway to be ready
    print("Gateway の準備が完了するまで待機中...")
    while True:
        status_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)

        current_status = status_response['status']
        print(f"Gateway ステータス: {current_status}")
        if current_status == 'READY':
            print(f"Gateway の最終詳細: {status_response}")
            break

        time.sleep(10)

    print("Gateway の準備が完了しました")
    return gateway_id, gateway_url, role_name

gateway_id, gateway_url, gateway_role_name = create_gateway_with_authcode()
print(f"\n Gateway 作成が完了しました: Gateway ID {gateway_id}")
print(f"Gateway URL: {gateway_url}")

## AgentCore Gateway でターゲットを作成
**/userInfo** 用のツールを含む Linkedin OpenAPI 仕様を使用して、AgentCore Gateway でターゲットを作成します。

In [ ]:
linkedin_openapi_spec = {
  "openapi": "3.0.0",
  "info": {
    "title": "LinkedIn UserInfo API",
    "version": "2.0.0"
  },
  "servers": [
    {
      "url": "https://api.linkedin.com/v2"
    }
  ],
  "paths": {
    "/userinfo": {
      "get": {
        "operationId": "getUserInfo",
        "summary": "Get User Information",
        "security": [
          {
            "BearerAuth": []
          }
        ],
        "responses": {
          "200": {
            "description": "Successful response",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/UserInfo"
                },
                "example": {
                  "sub": "782bbtaQ",
                  "name": "John Doe",
                  "given_name": "John",
                  "family_name": "Doe",
                  "picture": "https://media.licdn-ei.com/dms/image/C5F03AQHqK8v7tB1HCQ/profile-displayphoto-shrink_100_100/0/",
                  "locale": "en-US",
                  "email": "doe@email.com",
                  "email_verified": True
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "UserInfo": {
        "type": "object",
        "properties": {
          "sub": {
            "type": "string"
          },
          "name": {
            "type": "string"
          },
          "given_name": {
            "type": "string"
          },
          "family_name": {
            "type": "string"
          },
          "picture": {
            "type": "string"
          },
          "locale": {
            "type": "string"
          },
          "email": {
            "type": "string"
          },
          "email_verified": {
            "type": "boolean"
          }
        }
      }
    },
    "securitySchemes": {
      "BearerAuth": {
        "type": "http",
        "scheme": "bearer"
      }
    }
  }
}

In [ ]:
DEFAULT_RETURN_URL = "http://localhost:3021" # using some dummy URL as default. We will override when making JSON RPC calls

def create_linkedin_target(gatewayId, provider_arn):
    gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
    credentialProviderConfig = {
        "credentialProviderType": "OAUTH",
        "credentialProvider": {
            "oauthCredentialProvider": {
                "providerArn": provider_arn,
                "grantType": "AUTHORIZATION_CODE",
                "defaultReturnUrl": DEFAULT_RETURN_URL,
                "scopes": ["openid", "profile", "email"]
            }
        }
    }
    target_config = {
        "mcp": {
            "openApiSchema": {
                "inlinePayload": json.dumps(linkedin_openapi_spec)
            }
        }
    }
    try:
        response = gateway_client.create_gateway_target(
            name = "LinkedInAuthCode",
            description = "Target created for testing",
            credentialProviderConfigurations = [credentialProviderConfig],
            targetConfiguration= target_config,
            gatewayIdentifier=gatewayId
        )
        targetId = response["targetId"]
        print(f"Gateway {gatewayId} に Linkedin ターゲット {targetId} を作成しました")
        return targetId
    except Exception as e:
        print(e)


gateway_target_id = create_linkedin_target(gatewayId=gateway_id,
                                           provider_arn=target_cred_provider_arn)

## OAuth2 認可 URL セッションバインディングプロセス

ゲートウェイが作成されたので、先に進む前にセッションバインディングプロセスを理解しましょう。

OAuth2 認可 URL セッションバインディングプロセスは、OAuth2 認可セッションが AgentCore Identity の認証済みユーザーに適切に関連付けられることを保証する重要なセキュリティメカニズムです。このプロセスにより、セッションハイジャックを防止し、OAuth トークンが意図したユーザーにのみ付与されることを保証します。

参照：https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html

### セッションバインディングの仕組み
<div style="text-align:center">
    <img src="images/identity-session-binding.png" width="90%"/>
</div>

1. エージェントの呼び出し – エージェントコードまたは MCP クライアントは、元のエージェントユーザーが所有するアプリケーションまたはリソースにアクセスしたい場合、GetResourceOauth2Token API を呼び出して認可 URL を取得します。

2. 認可 URL の生成 – AgentCore Identity は、ユーザーがナビゲートしてアクセスに同意するための認可 URL とセッション URI を生成します。

3. 認可とアクセストークンの取得 – ユーザーは認可 URL にナビゲートし、エージェントがリソースにアクセスすることに同意します。その後、AgentCore Identity はユーザーのブラウザを、認可リクエストの元のユーザーに関する情報を含む HTTPS アプリケーションエンドポイントにリダイレクトします。この時点で、HTTPS アプリケーションエンドポイントは、元のエージェントユーザーがアプリケーションの現在ログインしているユーザーと同じであるかどうかを判断します。一致する場合、アプリケーションエンドポイントは CompleteResourceTokenAuth を呼び出し、AgentCore Identity がアクセストークンを取得して保存できるようにします。

4. エージェントを再呼び出ししてアクセストークンを取得 – アプリケーションが有効なレスポンスを返すと、エージェントアプリケーションは、ユーザー向けに元々リクエストされた OAuth2.0 アクセストークンを取得できます。ユーザーが一致しない場合、アプリケーションは何もしないか、試行をログに記録します。

アプリケーションエンドポイントがユーザー ID を検証できるようにすることで、AgentCore Identity はエージェントアプリケーションが、認可リクエストを開始したユーザーとアクセスに同意したユーザーが常に同じであることを保証できます。

### 環境対応 OAuth2 コールバックサーバー

このチュートリアルでは、異なる実行環境に自動的に適応する環境対応の `oauth2_callback_server.py` を使用します：

#### **ローカル開発：**
- **外部コールバック URL**：`http://localhost:9090/oauth2/callback`（ブラウザからアクセス可能）
- **内部通信**：`http://localhost:9090`（ノートブック ↔ サーバー）
- **サーバーバインディング**：`127.0.0.1`（localhost のみ、セキュア）

#### **SageMaker Workshop Studio：**
- **外部コールバック URL**：`https://<domain>.studio.<region>.sagemaker.aws/proxy/9090/oauth2/callback`（プロキシ経由でブラウザからアクセス可能）
- **内部通信**：`http://localhost:9090`（同じコンテナ内のノートブック ↔ サーバー）
- **サーバーバインディング**：`0.0.0.0`（SageMaker プロキシがサーバーに到達可能）

OAuth2 コールバックサーバーは `/opt/ml/metadata/resource-metadata.json` をチェックして環境を自動検出し、適切に設定します。

### oauth2_callback_server.py の機能

1. **ローカル FastAPI サーバーの実行**（ポート 9090）
   - ヘルスチェック用の `/ping` エンドポイントを提供
   - ユーザートークンを保存するための `/userIdentifier/token` エンドポイントを提供
   - OAuth リダイレクトを処理するための `/oauth2/callback` エンドポイントを提供

2. **ユーザートークンストレージの管理**
   - Cognito 認証からのユーザーの JWT トークンを保存
   - OAuth セッションを正しいユーザー ID に関連付け

3. **OAuth コールバック処理**
   - `session_id` パラメータ付きの OAuth リダイレクトを受信
   - セッションをバインドするために `CompleteResourceTokenAuth` を呼び出し
   - フローを完了する前にユーザー ID を検証

4. **セッションセキュリティの提供**
   - OAuth セッションが認証済みユーザーにバインドされていることを保証
   - OAuth トークンへの不正アクセスを防止

5. **環境検出**
   - ローカル環境と SageMaker Studio 環境を自動検出
   - URL とサーバーバインディングを適切に設定

### セキュリティに関する考慮事項

OAuth2 セッションバインディングプロセスには、いくつかのセキュリティ対策が含まれています：
- **URL 検証**：事前登録されたコールバック URL のみが許可
- **セッション検証**：トークン完了前にユーザーセッションを検証する必要あり
- **ユーザー ID バインディング**：OAuth セッションは認証済みユーザーに明示的にバインド
- **トークン分離**：各ユーザーの OAuth トークンは分離されセキュア
- **環境対応 URL**：各環境に適切な URL を自動的に使用

この包括的なアプローチにより、ローカルで実行する場合でも SageMaker Workshop Studio で実行する場合でも、OAuth2 フローがセキュアであり、マルチユーザー環境で正しいユーザーに適切に帰属することが保証されます。

---

In [ ]:
# Helper function to make MCP calls
def invoke_mcp(gatewayUrl, access_token, tool_params, method = "tools/call", protocol_version = '2025-11-25'):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
        'MCP-Protocol-Version': protocol_version
    }

    payload = {
        "jsonrpc": "2.0",
        "id": 24,
        "method": method,
        "params": tool_params
    }

    try:
        response = requests.post(gatewayUrl, headers=headers, json=payload)
        request_id = response.headers.get('x-amzn-requestid') or response.headers.get('x-amz-request-id')
        print("\nAmazon リクエスト ID:", request_id)
        response.raise_for_status()
        print(f"MCP 呼び出しステータスコード: {response.status_code}")
        print("レスポンス:")
        #if method != "tools/list":
        print(json.dumps(response.json(), indent=2))
        return response.json()

    except requests.exceptions.RequestException as e:
        print("エラー:", e)
        if hasattr(e, 'response') and e.response is not None:
            try:
                print("エラーレスポンス:", json.dumps(e.response.json(), indent=2))
            except:
                print("エラーレスポンステキスト:", e.response.text)
        raise

## MCP クライアントでテスト
次に、MCP クライアントで AgentCore Gateway をテストします。ユーザー同意フローをテストできるように、**forceAuthentication = True** メタデータで開始します。このステップを初めて実行する場合、**forceAuthentication = False** でも動作は同じです。

Elicitation レスポンスが返されます。これは URL を開いてユーザー同意を提供することを示しています。

完了すると、コールバックサーバーが AgentCore Identity とのセッションバインディングを処理します。

<div style="text-align:center">
    <img src="images/elicitation-resp.png" width="90%"/>
</div>

In [ ]:
import subprocess
from oauth2_callback_server import store_token_in_oauth2_callback_server, wait_for_oauth2_server_to_be_ready, get_oauth2_callback_base_url

full_scope = f"{RESOURCE_SERVER_ID}/{gateway_target_name}"
jwt_token = utils.get_token(user_pool_id=gw_user_pool_id, 
                            client_id=gw_client_id, 
                            client_secret=gw_client_secret, 
                            scope_string=full_scope, 
                            REGION=REGION)
bearer_token = jwt_token['access_token']

#Starting oAuth callback server
oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", REGION]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

successfully_started_oauth2_server = wait_for_oauth2_server_to_be_ready()
if not successfully_started_oauth2_server:
    print("セッションバインディングを処理するための OAuth2 コールバックサーバーの起動に失敗しました"
          "（https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html）")
else:
    store_token_in_oauth2_callback_server(bearer_token)

    #Test invoke tool
    CUSTOM_RETURN_URL = get_oauth2_callback_base_url() + "/oauth2/callback"
    print(f"コールバック URL を使用: {CUSTOM_RETURN_URL}")

    _meta = {
        "aws.bedrock-agentcore.gateway/credentialProviderConfiguration": {
            "oauthCredentialProvider": {
                "returnUrl": CUSTOM_RETURN_URL,
                "forceAuthentication": True
            }
        }
    }

    print("\n強制再認証をテスト中")
    resp = invoke_mcp(gatewayUrl= gateway_url, 
                      access_token=bearer_token,
                      tool_params={
                          "name": f"LinkedInAuthCode___getUserInfo",
                          "arguments": {"domainName": "integrals-dev-ed"},
                          "_meta": _meta
                        },
                      method="tools/call")

In [ ]:
#Invoking tool again after completion of oAuth flow
_meta = {
    "aws.bedrock-agentcore.gateway/credentialProviderConfiguration": {
        "oauthCredentialProvider": {
            "returnUrl": CUSTOM_RETURN_URL,
            "forceAuthentication": False
        }
    }
}
print("OAuth フロー完了後にツールを再呼び出し中")
resp = invoke_mcp(gatewayUrl= gateway_url, 
                  access_token=bearer_token,
                  tool_params={
                      "name": f"LinkedInAuthCode___getUserInfo",
                      "arguments": {"domainName": "integrals-dev-ed"},
                      "_meta": _meta
                    },
                  method="tools/call")

In [ ]:
oauth2_callback_server_process.terminate()

## リソースのクリーンアップ

このチュートリアルで作成されたすべてのリソースを削除するには、以下のセルを実行します：

1. AgentCore Gateway とターゲット
2. AgentCore Identity 資格情報プロバイダー
3. Amazon Cognito User Pool とクライアント
4. IAM Role

In [ ]:
# Delete AgentCore Gateway and all targets
print("ステップ 1: AgentCore Gateway リソースをクリーンアップ中...")
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
agentcore_cleanup = utils.delete_gateway(
    gateway_client=gateway_client,
    gatewayId=gateway_id
)

# Delete AgentCore Identity Credential Provider
print("\nステップ 2: AgentCore Identity 資格情報プロバイダーをクリーンアップ中...")
credential_cleanup = identity_client.delete_oauth2_credential_provider(
    name=target_cred_provider_name
)

# Delete Cognito User Pool
print("\nステップ 3: Cognito User Pool をクリーンアップ中...")
cognito_cleanup = utils.delete_cognito_user_pool(
    user_pool_id=gw_user_pool_id,
    region=REGION
)

# Delete IAM Role
print("\nステップ 4: IAM ロールをクリーンアップ中...")
iam_cleanup = utils.delete_iam_role(
    role_name=f'BedrockAgentCoreGatewayRole-{timestamp}'
)

## まとめ
Linkedin ツールをターゲットとして使用し、AgentCore Gateway で新しくリリースされた Authorization Code Grant ベースのアウトバウンド OAuth フローを正常にテストしました。